In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import altair as alt
import squarify
import geopandas as gpd
import json
from collections import defaultdict
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
data_2022 = pd.read_csv("Data/2022.csv", usecols=['INCIDENT_NUMBER', 'DISTRICT', 'CATEGORY'], dtype={'INCIDENT_NUMBER':str, 'DISTRICT':str, 'CATEGORY':str})
rox_2022 = data_2022.loc[data_2022['DISTRICT'] == 'B2']
fen_2022 = data_2022.loc[data_2022['DISTRICT'] == 'D4']
df_rox_2022 = rox_2022.groupby(['CATEGORY'])['INCIDENT_NUMBER'].count()
df_rox_2022 = df_rox_2022.sort_values(ascending=False)
rox_values_2022_sorted = squarify.normalize_sizes(df_rox_2022, 1000, 1000)
df_fenway_2022 = fen_2022.groupby(['CATEGORY'])['INCIDENT_NUMBER'].count()
df_fenway_2022_sorted = df_fenway_2022.sort_values(ascending=False)
fenway_values_2022_sorted = squarify.normalize_sizes(df_fenway_2022, 1000, 1000)

In [3]:
output_json = pd.DataFrame(squarify.padded_squarify(sizes=rox_values_2022_sorted, x=0, y=0, dx=1000, dy=1000))
output_json.insert(0, "Incident Count", df_rox_2022.array)
output_json.insert(0, "Category", df_rox_2022.keys())
output_json["x2"] = output_json["x"] + output_json["dx"]
output_json["y2"] = output_json["y"] + output_json["dy"]


In [4]:
alt.Chart(output_json).mark_rect().encode(
    x=alt.X('x:Q', axis=None),
    x2='x2:Q',
    y=alt.Y('y:Q', axis=None),
    y2='y2:Q',
    color=alt.Color("Category", type="nominal", sort=alt.EncodingSortField(field="Incident Count", order="descending"))
).properties(
    width=1000,
    height=1000
)

alt.Chart(...)

In [5]:
capital_plan = pd.read_excel('Data/23R Capital Plan 4.13.22.xlsx')
boston_map = gpd.read_file("Data/city-council-districts.geojson")

single_selection = alt.selection_point()

## bugfix taken from https://github.com/altair-viz/altair/issues/1612
boston_map.crs = {'init' :'epsg:27700'}
boston_map = boston_map.to_crs({'init': 'epsg:4326'})
choro_json = json.loads(boston_map.to_json())
boston_chart = alt.Chart(boston_map).mark_geoshape(
    strokeWidth=1,
    fill='none',
    fillOpacity=0,
    stroke='black'
).encode().properties(
    width=800,
    height=800
)

neighborhoods_map = gpd.read_file("Data/boston-neighborhoods.geojson");
neighborhoods_map.crs = {'init' :'epsg:27700'}
neighborhoods_map = neighborhoods_map.to_crs({'init': 'epsg:4326'})
choro_json = json.loads(neighborhoods_map.to_json())
neighborhoods_chart = alt.Chart(neighborhoods_map, title="Boston Districts by Total Budget").mark_geoshape(
    stroke='black',
    strokeWidth=.2,
).encode(
    color=alt.condition(single_selection, 'Name:N', alt.value('lightgray'))
).properties(
    width=800,
    height=800
).add_params(
    single_selection
);

base = alt.Chart(capital_plan).mark_bar().encode(
    x='Neighborhood',
    y='sum(Total_Project_Budget):Q',
).encode(
    color=alt.condition(single_selection, 'Neighborhood', alt.value('lightgray')),
    tooltip=alt.Tooltip("sum(Total_Project_Budget)", format="$,.2f"),
).add_params(
    single_selection
);

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
alt.vconcat(neighborhoods_chart + boston_chart, base)

In [48]:
# create dictionary mapping offense group to offense codes for csv that do not contain offense groups
df = pd.read_csv('Data/crime 2018.csv')

offense_group_dict = dict()

for code in df['OFFENSE_CODE'].unique():
    # get a row with the corresponding code
    single_row = df.loc[df['OFFENSE_CODE'] == code].iloc[0]
    group = single_row['OFFENSE_CODE_GROUP']
    
    offense_group_dict[code] = group

In [62]:
def extract_crime_comparison(early_crime, later_crime):
    if (type(early_crime) == str):
        early_crime = pd.read_csv(early_crime,
                                  usecols=['INCIDENT_NUMBER', 'DISTRICT', 'OFFENSE_CODE_GROUP'],
                                  dtype={'INCIDENT_NUMBER':str, 'DISTRICT':str, 'OFFENSE_CODE_GROUP':str})
    if (type(later_crime) == str):
        later_crime = pd.read_csv(later_crime,
                                  usecols=['INCIDENT_NUMBER', 'DISTRICT', 'OFFENSE_CODE_GROUP'],
                                  dtype={'INCIDENT_NUMBER':str, 'DISTRICT':str, 'OFFENSE_CODE_GROUP':str})
    
    # get early roxbury data
    early_crime = early_crime.loc[early_crime['DISTRICT'] == 'B2']
    early_crime = early_crime.groupby(['OFFENSE_CODE_GROUP'])['INCIDENT_NUMBER'].count()
    early_crime = early_crime.sort_values(ascending=False)
    
    # print(early_crime)
    
    # get later roxbury data
    later_crime = later_crime.loc[later_crime['DISTRICT'] == 'B2']
    later_crime = later_crime.groupby(['OFFENSE_CODE_GROUP'])['INCIDENT_NUMBER'].count()
    later_crime = later_crime.sort_values(ascending=False)
    

    
    net_change = pd.DataFrame(later_crime - early_crime)
    crime_type_list = ['Aggravated Assault',
                       'Larceny',
                       'Missing Person Reported',
                       'Auto Theft',
                       'Harassment', 
                       'Homicide',
                       'Robbery',
                       'Residential Burglary',
                       'Property Lost']
    net_change = net_change.loc[net_change.index.isin(crime_type_list)]
    net_change = net_change.reset_index()
    net_change.columns = ['CATEGORY', 'Net Change']
    
    return net_change


In [52]:
# need to format OFFENSE_CODE_GROUP -> doesn't exist in 2019, 2020, 2021

crime_2019 = pd.read_csv('Data/crime 2019.csv')
crime_2019['OFFENSE_CODE_GROUP'] = df['OFFENSE_CODE'].map(offense_group_dict)

crime_2020 = pd.read_csv('Data/crime 2020.csv')
crime_2020['OFFENSE_CODE_GROUP'] = df['OFFENSE_CODE'].map(offense_group_dict)

crime_2021 = pd.read_csv('Data/crime 2021.csv')
crime_2021['OFFENSE_CODE_GROUP'] = df['OFFENSE_CODE'].map(offense_group_dict)

/var/folders/71/723ncf_d5zl3dj94vpv18m780000gn/T/ipykernel_44816/2852634929.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_2019 = pd.read_csv('Data/crime 2019.csv')
/var/folders/71/723ncf_d5zl3dj94vpv18m780000gn/T/ipykernel_44816/2852634929.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_2020 = pd.read_csv('Data/crime 2020.csv')
/var/folders/71/723ncf_d5zl3dj94vpv18m780000gn/T/ipykernel_44816/2852634929.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_2021 = pd.read_csv('Data/crime 2021.csv')


In [63]:
change_2015_2016 = extract_crime_comparison('Data/crime 2015.csv', 'Data/crime 2016.csv')
change_2016_2017 = extract_crime_comparison('Data/crime 2016.csv', 'Data/crime 2017.csv')
change_2017_2018 = extract_crime_comparison('Data/crime 2017.csv', 'Data/crime 2018.csv')
change_2018_2019 = extract_crime_comparison('Data/crime 2018.csv', crime_2019)
change_2019_2020 = extract_crime_comparison(crime_2019, crime_2020)
change_2020_2021 = extract_crime_comparison(crime_2020, crime_2021)

is string
is string
is string
is string


In [66]:
selection = alt.selection_point()

# 2015 to 2016
change_from_2015 = alt.Chart(change_2015_2016).mark_bar(point=False).encode(
    x='CATEGORY',
    y='Net Change',
    color = alt.Color('Net Change', scale=alt.Scale(scheme="spectral", reverse=True))
).properties(
    title='Roxbury change in crime rates from 2015-2016'
).add_params(
    single_selection
)

# 2016 to 2017
change_from_2016 = alt.Chart(change_2016_2017).mark_bar(point=False).encode(
    x='CATEGORY',
    y='Net Change',
    color = alt.Color('Net Change', scale=alt.Scale(scheme="spectral", reverse=True))
).properties(
    title='Roxbury change in crime rates from 2016-2017'
).add_params(
    single_selection
)

# 2017 to 2018
change_from_2017 = alt.Chart(change_2017_2018).mark_bar(point=False).encode(
    x='CATEGORY',
    y='Net Change',
    color = alt.Color('Net Change', scale=alt.Scale(scheme="spectral", reverse=True))
).properties(
    title='Roxbury change in crime rates from 2017-2018'
).add_params(
    single_selection
)

# 2018 to 2019
change_from_2018 = alt.Chart(change_2018_2019).mark_bar(point=False).encode(
    x='CATEGORY',
    y='Net Change',
    color = alt.Color('Net Change', scale=alt.Scale(scheme="spectral", reverse=True))
).properties(
    title='Roxbury change in crime rates from 2018-2019'
).add_params(
    single_selection
)

# 2019 to 2020
change_from_2019 = alt.Chart(change_2019_2020).mark_bar(point=False).encode(
    x='CATEGORY',
    y='Net Change',
    color = alt.Color('Net Change', scale=alt.Scale(scheme="spectral", reverse=True))
).properties(
    title='Roxbury change in crime rates from 2019-2020'
).add_params(
    single_selection
)

# 2020 to 2021
change_from_2020 = alt.Chart(change_2020_2021).mark_bar(point=False).encode(
    x='CATEGORY',
    y='Net Change',
    color = alt.Color('Net Change', scale=alt.Scale(scheme="spectral", reverse=True))
).properties(
    title='Roxbury change in crime rates from 2020-2021'
).add_params(
    single_selection
)

In [67]:
# the net change legend doesn't represent negative values
change_from_2015 | change_from_2016 | change_from_2017

alt.HConcatChart(...)

In [65]:
change_from_2018 | change_from_2019 | change_from_2020

alt.HConcatChart(...)

In [15]:
## So basically, I don't think a line chart is going to reflect change the way we want it to--but we can totally
## create a series of visualizations like this from 2015->2022 which documents net changes in incidents over time
## as well as a year-to-year growth if we manipulate our data in a different way. 